In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [9]:
T = 8 # block size
positions = torch.arange(T)
print(positions)
print(positions[None,:].shape, positions[:, None].shape)
print(positions[None, :])
print(positions[:, None])

tensor([0, 1, 2, 3, 4, 5, 6, 7])
torch.Size([1, 8]) torch.Size([8, 1])
tensor([[0, 1, 2, 3, 4, 5, 6, 7]])
tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7]])


In [10]:
positions[None,:] - positions[:, None] # second gets broadcasted to become (T,T), then it's just element wise subtraction

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [-1,  0,  1,  2,  3,  4,  5,  6],
        [-2, -1,  0,  1,  2,  3,  4,  5],
        [-3, -2, -1,  0,  1,  2,  3,  4],
        [-4, -3, -2, -1,  0,  1,  2,  3],
        [-5, -4, -3, -2, -1,  0,  1,  2],
        [-6, -5, -4, -3, -2, -1,  0,  1],
        [-7, -6, -5, -4, -3, -2, -1,  0]])

In [11]:
block_size = T = 512

In [12]:
rel_bias = nn.Embedding(2 * block_size - 1, 1)
rel_bias

Embedding(1023, 1)

In [14]:
positions = torch.arange(T)
rel = positions[None, :] - positions[:, None]
rel

tensor([[   0,    1,    2,  ...,  509,  510,  511],
        [  -1,    0,    1,  ...,  508,  509,  510],
        [  -2,   -1,    0,  ...,  507,  508,  509],
        ...,
        [-509, -508, -507,  ...,    0,    1,    2],
        [-510, -509, -508,  ...,   -1,    0,    1],
        [-511, -510, -509,  ...,   -2,   -1,    0]])

In [15]:
max_rel = block_size - 1
print(max_rel)

rel_clipped = rel.clamp(-max_rel, max_rel)
rel_clipped

511


tensor([[   0,    1,    2,  ...,  509,  510,  511],
        [  -1,    0,    1,  ...,  508,  509,  510],
        [  -2,   -1,    0,  ...,  507,  508,  509],
        ...,
        [-509, -508, -507,  ...,    0,    1,    2],
        [-510, -509, -508,  ...,   -1,    0,    1],
        [-511, -510, -509,  ...,   -2,   -1,    0]])

In [16]:
rel_indices = rel_clipped + max_rel
rel_indices

tensor([[ 511,  512,  513,  ..., 1020, 1021, 1022],
        [ 510,  511,  512,  ..., 1019, 1020, 1021],
        [ 509,  510,  511,  ..., 1018, 1019, 1020],
        ...,
        [   2,    3,    4,  ...,  511,  512,  513],
        [   1,    2,    3,  ...,  510,  511,  512],
        [   0,    1,    2,  ...,  509,  510,  511]])

In [19]:
bias = rel_bias(rel_indices)
bias.shape

torch.Size([512, 512, 1])

In [21]:
bias = bias.squeeze(-1)
bias.shape

torch.Size([512, 512])

In [25]:
attn = torch.rand(4, 512, 512)
v = torch.rand(4, 512, 96)
attn.shape, v.shape

(torch.Size([4, 512, 512]), torch.Size([4, 512, 96]))

In [27]:
(attn @ v).shape

torch.Size([4, 512, 96])